<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_016_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Exercise 0
import Bio
import requests


Prot_IDs=['P04406','P10599','P68871','P04637']
def getFasta(ID):


